In [1]:
from csv import reader
from collections import defaultdict
from itertools import chain, combinations

In [2]:
class TreeNode:
    def __init__(self, itemName, frequency, parentTreeNode):
        self.itemName = itemName
        self.count = frequency
        self.parent = parentTreeNode
        self.children = {}
        self.next = None

    def incrementCount(self, frequency):
        self.count += frequency

    def printTree(self, ind=1):
        print('  ' * ind, self.itemName, ' ', self.count)
        for child in list(self.children.values()):
            child.printTree(ind+1)

In [3]:
def readFile(fname):
    itemSetList = []
    frequency = []

    with open(fname, 'r') as file:
        csv_reader = reader(file)
        count = 1
        for line in csv_reader:
            line = list(filter(None, line))
            count += 1
            if count >= 100:
                return itemSetList, frequency
            itemSetList.append(line)
            frequency.append(1)
    return itemSetList, frequency

In [4]:
def buildTree(itemSetList, frequency, minSup):
    headerTable = defaultdict(int)

    for idx, itemSet in enumerate(itemSetList):
        for item in itemSet:
            headerTable[item] += frequency[idx]

    headerTable = dict((item, sup)
                       for item, sup in headerTable.items() if sup >= minSup)
    if(len(headerTable) == 0):
        return None, None

    for item in headerTable:
        headerTable[item] = [headerTable[item], None]

    fpTree = TreeNode('Null', 1, None)

    for idx, itemSet in enumerate(itemSetList):
        itemSet = [item for item in itemSet if item in headerTable]
        itemSet.sort(key=lambda item: headerTable[item][0], reverse=True)
        currentTreeNode = fpTree
        for item in itemSet:
            currentTreeNode = updateTree(
                item, currentTreeNode, headerTable, frequency[idx])

    return fpTree, headerTable

In [5]:
def updateHeaderTable(item, targetTreeNode, headerTable):
    if(headerTable[item][1] == None):
        headerTable[item][1] = targetTreeNode
    else:
        currentTreeNode = headerTable[item][1]

        while currentTreeNode.next != None:
            currentTreeNode = currentTreeNode.next
        currentTreeNode.next = targetTreeNode

In [6]:
def updateTree(item, treeNode, headerTable, frequency):
    if item in treeNode.children:
        treeNode.children[item].incrementCount(frequency)
    else:
        newItemTreeNode = TreeNode(item, frequency, treeNode)
        treeNode.children[item] = newItemTreeNode
        updateHeaderTable(item, newItemTreeNode, headerTable)

    return treeNode.children[item]


def ascendFPtree(TreeNode, prefixPath):
    if TreeNode.parent != None:
        prefixPath.append(TreeNode.itemName)
        ascendFPtree(TreeNode.parent, prefixPath)

In [7]:
def findPrefixPath(basePat, headerTable):

    treeNode = headerTable[basePat][1]
    condPats = []
    frequency = []
    while treeNode != None:
        prefixPath = []

        ascendFPtree(treeNode, prefixPath)
        if len(prefixPath) > 1:

            condPats.append(prefixPath[1:])
            frequency.append(treeNode.count)
        treeNode = treeNode.next
    return condPats, frequency

In [8]:
def mineTree(headerTable, minSup, preFix, freqItemList):
    sortedItemList = [item[0] for item in sorted(
        list(headerTable.items()), key=lambda p:p[1][0])]
    for item in sortedItemList:
        newFreqSet = preFix.copy()
        newFreqSet.add(item)
        freqItemList.append(newFreqSet)
        conditionalPattBase, frequency = findPrefixPath(item, headerTable)
        conditionalTree, newHeaderTable = buildTree(
            conditionalPattBase, frequency, minSup)
        if newHeaderTable != None:
            mineTree(newHeaderTable, minSup,
                     newFreqSet, freqItemList)
def powerset(s):
    return chain.from_iterable(combinations(s, r) for r in range(1, len(s)))

In [9]:
def getSupport(testSet, itemSetList):
    count = 0
    for itemSet in itemSetList:
        if(set(testSet).issubset(itemSet)):
            count += 1
    return count

In [10]:
def associationRule(freqItemSet, itemSetList, minConf):
    rules = []
    for itemSet in freqItemSet:
        subsets = powerset(itemSet)
        itemSetSup = getSupport(itemSet, itemSetList)
        for s in subsets:
            confidence = float(itemSetSup / getSupport(s, itemSetList))
            if(confidence > minConf):
                rules.append([set(s), set(itemSet.difference(s)), confidence])
    return rules

In [11]:
def getFrequencyFromList(itemSetList):
    frequency = [1 for i in range(len(itemSetList))]
    return frequency

In [12]:
def fpgrowthFromFile(fname, minSupRatio, minConf):
    itemSetList, frequency = readFile(fname)
    minSup = len(itemSetList) * minSupRatio
    fpTree, headerTable = buildTree(itemSetList, frequency, minSup)
    if(fpTree == None):
        print('No frequent item set')
    else:
        freqItems = []
        freqItemset = []
        mineTree(headerTable, minSup, set(), freqItems)
        for i in freqItems:
            if len(i) == 4:
                freqItemset.append(i)
        rules = associationRule(freqItemset, itemSetList, minConf)
        return freqItemset, rules

In [13]:
minSup = 0.03
minConf = 0.7
freqItemSet, rules = fpgrowthFromFile("Datasets/Goods_service_dataset.csv", minSup, minConf)
print("Frequent Itemsets of length 4:\n")
print(freqItemSet, "\n")
print("Association rules:\n")
for i in rules:
    print(i[0], "=>", i[1], "  Confidence:", i[2])

Frequent Itemsets of length 4:

[{'39', '38', '48', '170'}, {'39', '32', '38', '48'}, {'48', '32', '39', '41'}, {'48', '38', '41', '39'}] 

Association rules:

{'170'} => {'39', '38', '48'}   Confidence: 1.0
{'39', '170'} => {'38', '48'}   Confidence: 1.0
{'38', '170'} => {'39', '48'}   Confidence: 1.0
{'48', '170'} => {'39', '38'}   Confidence: 1.0
{'39', '38', '170'} => {'48'}   Confidence: 1.0
{'39', '48', '170'} => {'38'}   Confidence: 1.0
{'38', '48', '170'} => {'39'}   Confidence: 1.0
{'32', '38'} => {'39', '48'}   Confidence: 1.0
{'39', '32', '38'} => {'48'}   Confidence: 1.0
{'32', '38', '48'} => {'39'}   Confidence: 1.0
{'32', '41'} => {'39', '48'}   Confidence: 0.75
{'41', '32', '48'} => {'39'}   Confidence: 1.0
{'39', '32', '41'} => {'48'}   Confidence: 1.0
